In [1]:
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
df <- data.frame(label = c('a1', 'a2', 'a3', 'a4', 'a5'),
                 x1 = c(1, 2, 3, NA, NA), 
                 x2 = c(100, NA, 300, 400, NA))
df

label,x1,x2
a1,1,100
a2,2,NA
a3,3,300
a4,NA,400
a5,NA,NA


### Set index for df instead of integer index

In [3]:
row.names(df) <- df$label
df

,label,x1,x2
a1,a1,1,100
a2,a2,2,NA
a3,a3,3,300
a4,a4,NA,400
a5,a5,NA,NA


### Remove a column in df

In [4]:
df <- subset(df, select = -c(label))
df

,x1,x2
a1,1,100
a2,2,NA
a3,3,300
a4,NA,400
a5,NA,NA


### remove "rows" in a dataframe with a NA value in one of the columns

In [5]:
df1 = na.omit(df)
df1

,x1,x2
a1,1,100
a3,3,300


### remove "rows" in a dataframe with all NA

In [6]:
df %>% filter_all(any_vars(!is.na(.)))

x1,x2
1,100
2,NA
3,300
NA,400


### Get column names

In [7]:
# col_names = colnames(df)
col_names = names(df)
col_names

[1] "x1" "x2"

In [8]:
class(col_names)

[1] "character"

In [9]:
col_names[1]

[1] "x1"

In [10]:
col_names[2]

[1] "x2"

In [11]:
split_chunk <- function(x, n) split(x, sort(rep(letters[1:n], each=n, len=length(x))))

x <- 1:10
split_chunk(x, 3)

$a
[1] 1 2 3 4

$b
[1] 5 6 7

$c
[1]  8  9 10

In [12]:
n <- 3
# xs = split(x, sort(x%%n))
# xs = split(x, rep_len(1:n, length(x)))
# xs = split(x, 1:n)  # divide it into 5 parts
#chunk = function(x, n) split(x, sort(rank(x) %% n))
#xs = chunk(x, n)
xs <- split(x, sort(rep_len(1:n, length(x))))
xs

$`1`
[1] 1 2 3 4

$`2`
[1] 5 6 7

$`3`
[1]  8  9 10

In [13]:
class(xs)

[1] "list"

In [17]:
class(xs[1][1])

[1] "list"

In [19]:
xs[1]

$`1`
[1] 1 2 3 4

In [20]:
unlist(xs[1], use.names = FALSE)

[1] 1 2 3 4

In [ ]:
ll = c(1, 2, 3)
class(ll)

In [ ]:
vec = unlist(xs[1])
vec

In [ ]:
class(vec)

In [ ]:
c(vec)

In [ ]:
# Creating a list.
my_list <- list(l1 = c(1, 3, 5, 7),                
                l2 = c(1, 2, 3),                    
                l3 = c(1, 1, 10, 5, 8, 65, 90))  
my_list

In [ ]:
vec1 = my_list['l1']
vec1

## II.

In [21]:
df <- data.frame(label = c('AA', 'BB', 'AA', 'CC', 'BB'),
                 x1 = c(1, 2, 3, NA, NA), 
                 x2 = c(100, NA, 300, 400, NA))
df

label,x1,x2
AA,1,100
BB,2,NA
AA,3,300
CC,NA,400
BB,NA,NA


In [22]:
labels = unique(df$label)
labels

[1] AA BB CC
Levels: AA BB CC